testing keras custom layer stuff

In [202]:
# import packages:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import os 
import tensorflow as tf
from tensorflow.keras import Input, Model, regularizers, constraints, layers, optimizers
from keras.layers import Layer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

define classes:

In [203]:
# randomizer seed:
np.random.seed(0)

# need to define a constraint for training the parameters:
class OrderedConstraint(constraints.Constraint):
    # constructor:
    def __init__(self):
        pass

    # call function for constraint:
    def __call__(self, W):
        return tf.sort(W, axis = 2)

# first layer: 
class MF_Layer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(MF_Layer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs

        # need to initialize antecedent parameters, but a <= b <= c. therefore 
        # generate a set of "raw parameters" to be sorted. these are not trained

        raw_params = np.random.uniform(low = 0.0, high = 50.0, size = (self.num_inputs, self.num_mfs, 3))
        sorted_params = tf.sort(raw_params, axis = -1)

        self.mf_params = self.add_weight(
            shape = (self.num_inputs, self.num_mfs, 3),             # num_inputs, num_mfs per input, 3 params per mf (triangular)
            initializer = tf.constant_initializer(sorted_params.numpy()),
            trainable = True,
            constraint = OrderedConstraint(),
            name = 'Antecedent_Params'
        )

    # custom setting of weights:
    def set_weights(self, params):
        # this function is used to set weights based on what a user provides
        # user must provide weights in the form of a np.array of shape (num_mfs, num_params)

        if params.shape != (self.num_inputs, self.num_mfs, 3):
            raise ValueError(f'Parameters provided are not of correct shape, expected ({self.num_inputs}, {self.num_mfs}, 3)')

        self.mf_params = params
        

    # function call:
    def call(self, inputs):
        # need to initialize the membership values:
        membership_values = []

        # for every input:
        for i in range(self.num_inputs):
            # get the memberships for that input:
            input_mf_params = self.mf_params[i]

            # need to now compute the fuzzified value for each membership function:
            fuzzified_values = []

            # for every membership function:
            for j in range(self.num_mfs):
                a = input_mf_params[j, 0]
                b = input_mf_params[j, 1]
                c = input_mf_params[j, 2]

                # check to see if we are on the edges:
                is_left_edge = (j == 0) & (a == b)                  # this would be the left-most ramp
                is_right_edge = (j == self.num_mfs - 1) & (b == c)  # this would be the right-most ramp
        
                # ramp calculations:
                left = tf.where((inputs[:, i] == a) & is_left_edge, 1.0, (inputs[:, i] - a) / (b - a))
                right = tf.where((inputs[:, i] == c) & is_right_edge, 1.0, (c - inputs[:, i]) / (c - b))

                output = tf.maximum(0.0, tf.minimum(left, right))

                fuzzified_values.append(output)
        
            # need to now stack the mf values for that given input:
            membership_values.append(tf.stack(fuzzified_values, axis = -1))

        # stack everything and return:
        return tf.stack(membership_values, axis = 1)


values for making model:

model testing:

In [204]:
model = MF_Layer(3,3)
params = tf.constant(np.array([
    [  # Parameters for input 1
        [0.0, 0.0, 6.0],
        [5/6, 5.0, 55/6],
        [4.0, 10.0, 10.0]
    ],
    [  # Parameters for input 2
        [0.0, 0.0 , 15.0],
        [25/12, 12.5, 275/12],
        [10.0, 25.0, 25.0]
    ],
    [  # Parameters for input 3
        [0.0, 0.0, 30.0],
        [25/6, 25.0, 275/6],
        [15.0, 50.0, 50.0]
    ]
]), dtype = tf.float32)

print(f'weights before: \n{model.mf_params.numpy()}\n')

model.set_weights(params)
print(f'weights after: \n{model.mf_params}\n')

weights before: 
[[[27.440676  30.138168  35.759468 ]
  [21.18274   27.24416   32.294704 ]
  [21.87936   44.58865   48.18314  ]]

 [[19.172075  26.444746  39.58625  ]
  [ 3.5518029 28.402227  46.27983  ]
  [ 1.0109199  4.356465  41.630993 ]]

 [[38.907837  43.500607  48.930916 ]
  [23.073969  39.02646   39.957928 ]
  [ 5.913721   7.1676645 31.996052 ]]]

weights after: 
[[[ 0.         0.         6.       ]
  [ 0.8333333  5.         9.166667 ]
  [ 4.        10.        10.       ]]

 [[ 0.         0.        15.       ]
  [ 2.0833333 12.5       22.916666 ]
  [10.        25.        25.       ]]

 [[ 0.         0.        30.       ]
  [ 4.1666665 25.        45.833332 ]
  [15.        50.        50.       ]]]



In [205]:
# going to check edge cases:
input = tf.constant([[0, 0, 0]], dtype = tf.float32)
model(input)

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]], dtype=float32)>